In [1]:
!pip install pyspark


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
 .appName("MovieRecommender") \
 .master("spark://spark-master:7077") \
 .config("spark.jars.packages", "org.postgresql:postgresql:42.7.1") \
 .config("spark.driver.memory", "2g") \
 .config("spark.executor.memory", "3g") \
 .config("spark.driver.host", "recommender-jupyter") \
 .config("spark.driver.bindAddress", "0.0.0.0") \
 .getOrCreate()

jdbc_url = "jdbc:postgresql://postgres:5432/recommender"
properties = {
    "user": "recommender",
    "password": "recommender",
    "driver": "org.postgresql.Driver"
}

In [5]:
# Load tables
ratings = spark.read.jdbc(
    jdbc_url, "movielens.ratings", properties=properties,
    column="user_id", lowerBound=1, upperBound=300000, numPartitions=10
)
movies = spark.read.jdbc(jdbc_url, "movielens.movies", properties=properties)

ratings.show(5)
movies.show(10)

[Stage 1:>                                                          (0 + 1) / 1]

+-------+--------+------+-------------------+
|user_id|movie_id|rating|   rating_timestamp|
+-------+--------+------+-------------------+
|      1|       2|   3.5|2005-04-02 23:53:47|
|      1|      29|   3.5|2005-04-02 23:31:16|
|      1|      32|   3.5|2005-04-02 23:33:39|
|      1|      47|   3.5|2005-04-02 23:32:07|
|      1|      50|   3.5|2005-04-02 23:29:40|
+-------+--------+------+-------------------+
only showing top 5 rows
+--------+--------------------+--------------------+
|movie_id|               title|              genres|
+--------+--------------------+--------------------+
|       1|    Toy Story (1995)|Adventure|Animati...|
|       2|      Jumanji (1995)|Adventure|Childre...|
|       3|Grumpier Old Men ...|      Comedy|Romance|
|       4|Waiting to Exhale...|Comedy|Drama|Romance|
|       5|Father of the Bri...|              Comedy|
|       6|         Heat (1995)|Action|Crime|Thri...|
|       7|      Sabrina (1995)|      Comedy|Romance|
|       8| Tom and Huck (1995)| 

In [ ]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# Prepare data
(training, test) = ratings.randomSplit([0.7, 0.3], seed=42)

# Build ALS model
als = ALS(
    maxIter=10,
    regParam=0.1,
    userCol="user_id",
    itemCol="movie_id",
    ratingCol="rating",
    coldStartStrategy="drop"
)

model = als.fit(training)

# Evaluate
predictions = model.transform(test)
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)
rmse = evaluator.evaluate(predictions)
print(f"RMSE: {rmse}") # jak interpretować wartość

[Stage 24:=============================================>           (8 + 2) / 10]

In [ ]:
# Top 10 recommendations for each user
user_recs = model.recommendForAllUsers(10)
user_recs.show(5, truncate=False)

# jakie filmy ogladal wczesniej a jakie dostal rekomendacje

# Top 10 recommendations for a specific user
user_42_recs = model.recommendForUserSubset(
    spark.createDataFrame([(42,)], ["user_id"]), 10
)

# Join with movie titles
from pyspark.sql.functions import explode

user_42_recs_flat = user_42_recs.select(
    "user_id",
    explode("recommendations").alias("rec")
).select("user_id", "rec.movie_id", "rec.rating")

user_42_recs_flat.join(movies, "movie_id").show(truncate=False)

In [ ]:
from pyspark.sql.functions import explode, col

movie_recs = model.recommendForItemSubset(
    spark.createDataFrame([(1,)], ["movie_id"]), 10
)

# Check the schema
movie_recs.printSchema()

# Explode correctly - it's user_id, not movie_id
movie_recs_flat = movie_recs.select(
    col("movie_id"),
    explode("recommendations").alias("rec")
).select(
    "movie_id",
    col("rec.user_id").alias("user_id"),
    col("rec.rating").alias("score")
)

movie_recs_flat.show()

In [ ]:
from pyspark.ml.feature import Normalizer
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from pyspark.ml.linalg import Vectors
import numpy as np

# Get item factors
item_factors = model.itemFactors

# Get Toy Story's factor vector
toy_story_factor = item_factors.filter(col("id") == 1).select("features").collect()[0][0]

# Calculate cosine similarity
def cosine_sim(v):
    a = np.array(toy_story_factor)
    b = np.array(v)
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

cosine_udf = udf(cosine_sim, DoubleType())

# Find similar movies
similar_movies = item_factors \
    .withColumn("similarity", cosine_udf(col("features"))) \
    .filter(col("id") != 1) \
    .orderBy(col("similarity").desc()) \
    .limit(10) \
    .withColumnRenamed("id", "movie_id")

# Join with movie titles
similar_movies.join(movies, "movie_id") \
    .select("movie_id", "title", "similarity") \
    .show(truncate=False)